In [1]:
import math

# Link Budget

Mach 30 is designing a radio to receive transmissions from satellites (a ground station), named Ground Sphere. One of the most crucial calculations for determining whether the ground station's design will be able to receive the signals transmitted by a given satellite is the link budget. Simply put, a link budget factors in all gains and losses in the system composed of the transmitter (satellite) and receiver (ground station), and everything in between.

The calculations themselves are implemented by a python class named LinkBudget. The class models what are generally the most significant factors in calculating a link budget. Because there are many factors that are difficult or impossible to accurately factor in, such as the amount a given storm may introduce noise into the system, several aspects of the budget are averages or otherwise estimated values.

## Overview

This particular link analysis uses 13 inputs to calculate the Link Margin. These inputs are described in the table below. Please take note that for the equations to work, losses will be negative numbers, while gains are positive numbers.

| Input | Units |Symbols|
|---|---|-----:|
|Downlink Frequency| MHz |$${f}$$ | 
|Target Energy per Bit to Noise Power Ratio | dB |$${Eb}/{No_T}$$ |
|Implementation Loss of Transmitter | dB | $${L_{TrImp}}$$| 
|Transmit Power | W |$${P_{Tr}}$$ | 
|Transmit Losses | dB |$${L_{Tr}}$$| 
|Transmit Antenna Gain | dB |$${G_{Tr}}$$ |  
|Transmit Pointing Loss | dB |$${L_{TrP}}$$ |
|Link Distance | km |$${D}$$ | 
|Polarization Losses | dB | $${L_{Pol}}$$| 
|Atmospheric Loss | dB |$${L_{At}}$$ |
|Receiver Antenna Gain | dB |$${G_{Re}}$$ | 
|Receiver Pointing Loss | dB |$${L_{ReP}}$$ | 
|System Noise Figure | dB |$${N_{Sys}}$$ | 
|Noise Bandwidth | kHz|$$BW$$|

|Other Variables |Units|Symbols| 
|---|---|--:|
|Wavelength| m |$${\lambda}$$ |
|Required Eb/No|dB |$${{Eb/No}_R}$$ |
|Converted Transmit Power| dBm |$${P_{TrC}}$$ |
|Transmit EIRP| dB |$${EIRP}$$ |
|Downlink Path Loss| dB |$${L_{Path}}$$ |
|Received Power|dBm|$${P_{Re}}$$|
|Minimum Detectable Signal | dBm | $${MDS}$$| 
|Final Eb/No| dB | $${Eb/No}_{F}$$| 
|Link Margin | dB | $${Link_{Mar}}$$| 

The equation used will be Link Margin = Actual Eb/No - Required Eb/No, where Required Eb/No = Target Eb/No + Implementation Losses and Actual Eb/No = Received Power - MDS. The target Eb/No and the Implementation Losses are usually assumed based on previous experience and research. The Received Power = Transmit EIRP + Downlink Path Loss + Polarization Loss + Atmosphere Loss + Receive Antenna Gain + Receiver Pointing Losses, and the MDS = -174 + 10log(Noise Bandwidth * 1000) + System Noise Figure. Some of these numbers are inputs while others are derived from the inputs. How to derive these values is explained below. 

*<p style="text-align: center;"><b>Equations Listed in the Order Mentioned</b></p>*

$$Link_{Mar} [dB] = {{Eb/No}_F} [dB] - {{Eb/No}_R} [dB]$$

$$ {Eb/No}_F [dB] = {P_{Re}}[dBm] -{MDS}[dBm]$$

$${Eb/No}_R [dB] = {Eb/No}_T[dB] + {L_{TrImp}} [dB]$$

$$P_{Re} [dBm] = {EIRP}[dB] + L_{Path}[dB] + L_{Pol} [dB]+ L_{At} [dB]+ G_{Re}[dB] + L_{ReP}[dB]$$

$$MDS[dB] = -174 + 10log_{10}(BW[kHz]*1000) + N_{Sys}[dB]$$

## Downlink

The Downlink Wavelength = The Speed of Light / Downlink Frequency. The Target Eb/No and Implementation Loss, as well as the Required Eb/No are found through research based on your particular transmission and educated assumptions. The Target Eb/No for NOAA Automatic Picture Transmission (APT) reception is typically around 20 - 25 dB, however it varies based on communication scheme (e.g. modulation, forward error correction, coding schemes, etc.). The Implementation Loss is loss due deviation of a real system from theoretically perfect system in math. This typically assumed to be around 1 dB.


$$ \lambda [m] = c/f$$

$$ Eb/No_R [dB]= L_{Imp}[dB] + Eb/No_T[dB] $$


## Transmitter

Whenever information is sent from one place to another, such as when a satellite transmits an image down to Earth to be received by a radio on the ground (ground station), the power (gain) of the signal being transmitted from the satellite is a critical factor in how well it will be able to be received by the ground station. This section of the document serves as a guide to determining what the gain of the source of the transmission may be. In some cases, the gain may be known, but in other cases it must be estimated based on knowledge of the components of the radio system used to transmit.

In the event that you do not know what the gain of the transmitter is, there are a few methods one might use to determine it. Some satellites have their broadcast frequency publicly available, such as with the NOAA satellites. This may be on the organization's website, but a great source is the Earth Observation Portal's Satellite Mission Database, which contains detailed information about a wide number of satellites, all in one place.

If you cannot access this information, you may have to make some estimates based upon known equipment of the transmitter, or simply make a guess for the purposes of calculating your link budget. If you're attempting to calculate an estimate, knowing what the gain of the antenna, radio, and amplifier are is helpful, where you may then be able to use a conservative estimate for the loss of other components in the system. This estimate may be 1.0db to 1.5db of loss in the 100MHz range (or higher for longer lengths of transmission line). This will decrease in higher frequencies due to the need for components of the transmission system such as filters or waveguides to be tightly matched to the transmission frequency.

Find the power of the transmitter and convert it to dBm. To convert from watts to dBm, the conversion is power_in_dBm = 10log(power_in_watts*1000). The transmit losses, pointing losses, and antenna gain will be decided based on research, as mentioned above. The Transmit EIRP = Transmit Power + Transmit Losses + Transmit Antenna Gain + Pointing Losses.

$$ P_{TrC} [dBm] = 10log_{10}(P_{Tr}[W]*1000) $$

$$ EIRP[dB] = P_{TrC}[dB] + L_{Tr}[dB] + G_{Tr}[dB] + L_{TrP}[dB] $$

## Calculating Distance from Transmitter to Receiver

Now that the transmitter is sending data, it has to travel to the receiver. As light travels from the transmitter to the receiver, it disperses, spreading out into a larger area than that of the transmitter. As a result, the distance the transmission travels incurs loss in the signal. There is also loss accumulated from traveling through a given medium (air for example), which we will account for in the next section. Calculating the distance from the transmitter to receiver so that we can determine the resulting loss involves a little bit of basic trigonometry.

#### Known Variables ####
In the case of a satellite, the main piece of information you need to know is the orbital altitude of the satellite. Additionally, the altitude the receiver will be used at is important, but if unknown, the lowest altitude the receiver may be used at should be used (for example, sea level, or 0 meters). The last piece of the puzzle is the angle relative to the receiver the satellite will be as it rises above the horizon and the receiver starts to receive, as this is the angle at which the most signal loss will occur. 25° is a good value to use here, accounting for trees and other ground based obstructions. If you know the receiver is going to be used from some place without many ground based obstructions, such as a mountaintop, you might use a larger angle.

With that angle and the altitude of the satellite and receiver, we can calculate the slant range, or the distance between the transmitter to receiver, using basic trigonometry.  As the satellite travels over head, it is orbiting the Earth, and the receiver is on the ground. Both of these share the center of the Earth in common, acting as the third point of the triangle. Examining Diagram 1 below, you can see a visual representation of this, along with number of variables displayed. 

![alt text](diagram1.png "Diagram 1")
*<p style="text-align: center;">Diagram 1</p>*

The location of the transmitter (***A***), receiver (***B***), and center of the Earth (***C***) are shown, as well as the angle above the horizon the satellite will first be received from (***ϕ***). What needs to be solved for is the slant range, represented as ***c***. Some of these variables we have not yet assigned a value. The radius of the Earth (***r***) is known. Because the receiver is at sea level, this is also the distance from ***B*** to ***C***, represented in the diagram as ***a***. The satellite orbits Earth at a known altitude, and that plus ***r*** is equal to ***b***, or the distance from ***A*** to ***C***. If the receiver is not at sea level, ***a*** is similarly simply the altitude above sea level of the receiver plus ***r***

There are still some variables in the diagram that are unknown. These are ***β***, the angle at ***B*** between ***A*** and ***C***, ***θ***, the angle at ***C*** between ***A*** and ***B***, and similarly ***α***, the angle at ***A*** between ***B*** and ***C***. Lucky for us, just like ***b***, we have the information to calculate ***β***, with which we can calculate the other angles. Since ***A*** is at a known angle above the horizon, and the horizon is perpendicular to ***a***, ***β*** is the angle above the horizon + 90°.

#### Calculating Slant Range ####
To solve for the remaining values, including the slant range, we need only two equations. The first is the law of sines, which states that for a triangle, the sine of an angle divided by the length of the opposite side of the triangle is equal to the same for each other angle. Below is a representation of this equation.

$$\frac{sin(α)}{a} = \frac{sin(β)}{b} = \frac{sin(θ)}{c}$$

With this equation, we can determine the value for ***α*** given the information we already know. 

$$\frac{sin(α)}{a} = \frac{sin(β)}{b}$$
$$sin(α) = a * \frac{sin(β)}{b}$$
$$sin(α) = \frac{a}{b} * sin(β)$$
$$α = sin^{-1}\bigg(\frac{a}{b} * sin(β)\bigg)$$

The other equation is the fact that the sum of the inner angles of a triangle are equal to 180. Below is a representation of this equation. 

$$180 = α + β + θ$$

With this equation, when two of the angles are known, the third can be calculated using subtraction. This equation is subsequently used to calculate ***θ***. With ***θ*** known, law of sines again can be used to calculate ***c***.

We can now represent all of the information we have, as well as the equations to solve for the as yet unknowns. The table below lists all this information.

| Variable | Description | Equation to Solve for Variable |
|---|---|---|---|
| ***A*** | Transmitter / Satellite |   |
| ***B*** | Receiver |   |
| ***C*** | Center of Earth |   |
| ***r*** | Earth Radius |   |
| ***a*** | Distance from ***C*** to *** B *** | $$r + B_{altitude}$$ |
| ***b*** | Distance from ***C*** to *** A *** | $$r + A_{altitude}$$ |
| ***c*** | Slant Range | $$sin(θ) * \frac{b}{sin(β)}$$ |
| ***ϕ*** | Angle of Transmitter above Horizon |   |
| ***α*** | Angle at ***A*** between ***B*** and ***C*** | $$sin^{-1}\bigg(\frac{a}{b} * sin(β)\bigg)$$ |
| ***β*** | Angle at ***B*** between ***A*** and ***C*** | $$θ + 90°$$ |
| ***θ*** | Angle at ***C*** between ***A*** and ***B*** | $$180 - α - β$$ |

## Path and Atmospheric Losses

The Link Distance was solved for in the section above. Downlink Path Loss (dB) =  -1*10log((4PI*distance*1000/wavelength)^2). Polarization Loss is zero in this example because both of the antennas are right hand circularly polarized (RHCP). As a counterexample, if one of the antennas is circularly polarized and the other antennas is linearly polarized, the polarization loss would be around 3 [dB].

$$L_{Path} = (-1)*10log_{10}((\frac{4{\pi}*D*1000}{\lambda})^2)$$

Losses due to atmospheric gases (Nitrogen, Oxygen, Carbon Dioxide, Hydrogen, etc.) are nearly independent of atmospheric temperature, mean density and relative humidity at frequencies below 2 GHz.  Atmospheric absorption depends strongly upon the total number of molecules distributed along the path between the spacecraft and the ground station.  This, in turn, means that the losses from or to the satellite are elevation angle dependent.									
									
The data used here  is taken from "Radiowave Propagation in Satellite Communications" by									
Louis J. Ippolito, Jr., Van Nostrand-Reinhold, 1986, pp. 33-34, Tables 3-3a-c. If you are using uplink or downlink frequencies above 2 GHz, refer to the referenced text given above to determine the appropriate atmosperic losses.

The table below is a look-up table.


|Elevation Angle|Loss|Unit|
|---|---|--:|
|0$^{\circ}$|10.2|dB|
|2.5$^{\circ}$|4.6|dB|
|5$^{\circ}$|2.1|dB|
|10$^{\circ}$|1.1|dB|
|30$^{\circ}$|.4|dB|
|45$^{\circ}$|.3|dB|
|90$^{\circ}$|0.0|dB|

This information was taken from the AmSat-IARU Link Model.


## Receiver

The Receiver Antenna Gain will be based on the shape of the antenna and the Pointing Offset is affected by the angle it is receiving the transmission from. The Received Power = Transmit EIRP + Downlink Path Loss + Polarization Loss + Atmospheric Losses + Receiver Antenna Gain + Receiver Pointing Losses.

$$P_{Re}[dBm] = EIRP[dB] + L_{Path} [dB]+ L_{Pol}[dB] + L_{At} [dB]+ G_{Re}[dB] + L_{ReP}[dB] $$

## Solve for Link Margin

Now that all the bits and pieces have been solved for, plug them back into the equations listed at the beginning of the document to solve for the Link Margin.

$$MDS[dB] = -174 + 10log_{10}(BW[kHz]*1000) + N_{Sys}[dB]$$

$$P_{Re}[dB] = {EIRP}[dB] + L_{Path}[dB] + L_{Pol}[dB] + L_{At}[dB] + G_{Re}[dB] + L_{ReP}[dB]$$

$$ {Eb/No}_F[dB] = {P_{Re}}[dB] -{MDS}[dB]$$

$${Eb/No}_R[dB] = {Eb/No}_T[dB] + {L_{TrImp}}[dB]$$

$$Link_{Mar}[dB] = {{Eb/No}_F}[dB] - {{Eb/No}_R}[dB]$$